In [1]:
import json, iso8601, pprint, os, codecs

# Load, Clean, Sort, Convert GeoLocated data collected form Twitter (GNIP)

This notebook: 

1. Loads raw gnip data
1. Writes full GNIP GEOJSONL files per user

## [Spark Status](http://epic-analytics.cs.colorado.edu:4040/jobs/)

In [2]:
input_directory  = "/data/chime/Hurricanes_HIM/contextual-plus/maria-geo-pr/GNIP/"
output_directory = "/data/chime/geo2/MARIA-PR-CONTEXTUALPLUS/"

## Step 1. Load all the Tweets!

In [3]:
#Load the files
raw_strings = sc.textFile(input_directory)

#Filter out the duds
strings = raw_strings.filter(lambda x: x!="")

#JSONs
jsons  = strings.map(json.loads)

tweets = jsons.filter(lambda x: 'info' not in x.keys())

### Check that this is working so far

In [4]:
pprint.pprint(tweets.take(1)[0])

{u'actor': {u'displayName': u'PR Accounting',
            u'favoritesCount': 0,
            u'followersCount': 462,
            u'friendsCount': 315,
            u'id': u'id:twitter.com:856869560',
            u'image': u'https://pbs.twimg.com/profile_images/720870033031700480/K70PJyXS_normal.jpg',
            u'languages': [u'en'],
            u'link': u'http://www.twitter.com/tmj_ptr_acct',
            u'links': [{u'href': u'http://www.careerarc.com/job-seeker',
                        u'rel': u'me'}],
            u'listedCount': 16,
            u'location': {u'displayName': u'Puerto Rico',
                          u'objectType': u'place'},
            u'objectType': u'person',
            u'postedTime': u'2012-10-01T19:08:28.000Z',
            u'preferredUsername': u'tmj_ptr_acct',
            u'statusesCount': 21,
            u'summary': u'Follow this account for geo-targeted Accounting job tweets in Puerto Rico. Need help? Tweet us at @CareerArc!',
            u'twitterTimeZone':

## Step 1.5:  Expand collection to all matching rules before grouping On User

In [8]:
# new_rdd = 

test = tweets.filter(lambda t: len(t['gnip']['matching_rules'])>1)

#Going to have to ask Gerard more about this one...

# Example: [{u'id': 163901075970690121,
#      u'id_str': u'163901075970690121',
#      u'tag': u'1365318420:MaybeImNicolee:Nasty Gal\u26a1 \U0001f49b'},
#     {u'id': 2995764339274592720,
#      u'id_str': u'2995764339274592720',
#      u'tag': u'2840774861:skateromero21:Erick Romero \U0001f4f7'}],

type(test)
#tweets_gb_user = tweet_jsons.groupBy(lambda t: t['gnip']['matching_rules'][0]['tag'])

pyspark.rdd.PipelinedRDD

## Step 2: Group tweets by user

In [84]:
tweets_gb_user = tweets.groupBy(lambda t: t['actor']['id'])
tweets_gb_user.cache() #We should probably cache these? If we want to use them again?

PythonRDD[72] at RDD at PythonRDD.scala:53

#### Check on the status of this operation, should see a tuple of: `(user_id, iterable)`

In [85]:
pprint.pprint(tweets_gb_user.take(1)[0])

(u'id:twitter.com:98402043',
 <pyspark.resultiterable.ResultIterable object at 0x7f3cf2663790>)


In [86]:
def writeGeoJSON(uTuple):
    u, iterable = uTuple
    tweets = list(iterable)
    handle = tweets[0]['actor']['preferredUsername']
    tweets.sort(key=lambda t: t['postedTime'])
    
    #First, check for `geo`
    features = []
    geo_count = 0;
    
    for t in tweets:
        try:
            geo = t.get('geo',None)
            if geo:
                geo_count += 1;
                geo = {'type':"Point",'coordinates':list(reversed(geo['coordinates']))}
            feat = {
                'type':'Feature',
                'geometry': geo,
                'properties':{
                    'user':t['actor']['preferredUsername'],
                    'text':t['body'],
                    'date':t['postedTime'],
                    'tweetID': t['id'].split(":")[2]
                }
            }
            features.append(feat)
        except:
            raise
    
    #Minimum tweet count?
    if geo_count > 5:
        json.dump({'type':'FeatureCollection', 'features': features},
                  codecs.open(output_directory + handle+".geojson",'w'))        

## Step 3: Write out the `geojsonl` files

For now just writing the full GNIP files, but in the future, this can probably be streamlined?

In [87]:
if not os.path.exists(output_directory):
    os.mkdir(output_directory)

In [88]:
tweets_gb_user.foreach( writeGeoJSON )